<a href="https://colab.research.google.com/github/rcleveng/notebooks/blob/main/Complete_(No_DB)_Manual_RAG_with_Google_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple RAG langchain example using Google's Generate AI API.
This example let's you ask questions against a book.

You'll need an API key from Generative AI Studio.


In [ ]:
!pip install langchain google.generativeai langchain-google-genai ratelimit more_itertools


# Restart the kernel once dependencies are updated

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

Start Milvus vector database that is used for holding the vectors.

In [2]:
# @title Set the book URL to use for Q&A

# Change this to use a different book.
BOOK_URL="https://www.gutenberg.org/cache/epub/3300/pg3300.txt" #@param {type:"string"}

In [3]:
# @title Fetch and split URL into chunks

import requests
from google.colab import userdata

# Langchain does a reasonable job splitting easily, so we'll use that

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.schema import Document


# Get text from URL
def get_data(url):
 r = requests.get(url)
 return r.text

BOOK_TEXT = get_data(BOOK_URL)
doc =  Document(page_content=BOOK_TEXT, metadata={"source": "local"})
text_splitter = RecursiveCharacterTextSplitter (chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents([doc])

# Count the number of chunks
print ('Finished splitting document into ', len(texts), 'chunks.')


Finished splitting document into  3102 chunks.


In [ ]:
import pandas as pd
import google.generativeai as genai
from ratelimit import limits, sleep_and_retry
from google.colab import userdata
from more_itertools import batched


import requests

clean_texts = list(map(lambda x: x.page_content, texts))
google_api_key = userdata.get('PALM_KEY')
genai.configure(api_key=google_api_key)


Count = 1
@sleep_and_retry
@limits(calls=1500, period=60)
def batch_embed_fn(text):
  global Count
  Count += 1
  if Count % 100 == 0:
    print("Batch", Count)
  return genai.embed_content(model="models/embedding-001",
                             content=text,
                             task_type="retrieval_document")["embedding"]


embeddings = []
for batch in batched(clean_texts, 50):
  emb_batch = batch_embed_fn(batch)
  embeddings.extend(emb_batch)


d = { 'id' : range(0, len(clean_texts)),
      'text' : clean_texts,
      'embeddings' : embeddings }
df = pd.DataFrame(d)

print('Dataframe created for ', len(clean_texts), ' items.')


In [ ]:
df


In [ ]:
# @title Ask Questions on The Wealth of Nations
import numpy as np

# Ask question
question = "What is the title of the book?" #@param {type:"string"}

# emedded query
emb_question = genai.embed_content(model="models/embedding-001",
                                   content=question,
                                   task_type="retrieval_query")["embedding"]

print("emb_question: ", type(emb_question), emb_question)


# Compute cosine similarity
cosine_similarities = np.dot(embeddings, emb_question).flatten()

# Find the index of the most similar sentence
most_similar_index = np.argmax(cosine_similarities)
topk = np.argsort(cosine_similarities)[-10:]
for k in topk:
  print('\n\n***********************\nArticle id: ', k, ': ', clean_texts[k])

print(f"\n\n***********************\nMost similar sentence: {clean_texts[most_similar_index]}")



context = "\n".join(map(lambda k: clean_texts[k], topk))

prompt = f"""
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try
to make up an answer.

{context}

Question: {question}
Helpful Answer:

"""

print("\n\n***********************\nUsing full prompt: ", prompt)
answer = ""

model_name = 'models/text-bison-001' # @param ["gemini-pro", "models/text-bison-001"] {allow-input: false}

# emedded query
if model_name == "gemini-pro":
  model = genai.GenerativeModel('gemini-pro')
  answer = model.generate_content(question).text
else:
  c = genai.generate_text(model=model_name, prompt=question, max_output_tokens=2000)
  answer = c.result

print("answer: ", answer)
